In [1]:
import numpy as np
import pandas as pd

import pytwits
import requests

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as sentintentanalyzer

In [2]:
ticker_list = ["TSLA", "GOOG", "AAPL", "AMZN", "FB", "MSFT"]
tweets = []
tickers = []
access_token = '26c8e5b3d7bd2903f6091364ae85a91cce9b9aa4'
stocktwits = pytwits.StockTwits(access_token=access_token) # This would also work without passing token.

for ticker in ticker_list:
    symbol, cursor, messages = stocktwits.streams(path = 'symbol', id = ticker, limit = 60) # To set limit on messages (max = 30) -> limit = __)
    for msg in messages:
        tweets.append(msg.body)
        tickers.append(symbol.symbol)    

tweet_df = pd.DataFrame({'Ticker': tickers, 'Tweet': tweets})

In [3]:
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("&#39;", "'")
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("$", "")
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace('&quot;', '"')
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("&amp;", "&")

In [4]:
def sentiment(tweet):
    analyze = sentintentanalyzer()
    comp = analyze.polarity_scores(tweet)['compound']
    if comp > 0:
        return "Positive"
    elif comp < 0:
        return "Negative"
    else:
        return "Neutral"

def polarity(tweet):
    analyze = sentintentanalyzer()
    return analyze.polarity_scores(tweet)['compound']

In [5]:
tweet_df['Sentiment'] = tweet_df['Tweet'].apply(lambda x: sentiment(x))
tweet_df['Score'] = tweet_df['Tweet'].apply(lambda x: polarity(x))
tweet_df

,Ticker,Tweet,Sentiment,Score
0,TSLA,TSLA buy the dip - consolidation at this level...,Negative,-0.1280
1,TSLA,TSLA great little history of some long-time T...,Positive,0.2263
2,TSLA,TSLA *facepalm*,Neutral,0.0000
3,TSLA,Today trading plan ANF FB SQ ROKU AMD CRON CGC...,Neutral,0.0000
4,TSLA,TSLA looks like nobody has faith in all the Ap...,Positive,0.6486
5,TSLA,TSLA Every single day the news on Tesla gets w...,Negative,-0.8658
6,TSLA,"TSLA &lt;300, buy zone for me, personally.",Neutral,0.0000
7,TSLA,TSLA SPECIAL REPORT: IS TESLA a SHELL GAME? Mu...,Positive,0.2789
8,TSLA,"TSLA bought 200sh in mar 17 at 260, sold apr 1...",Positive,0.4404
9,TSLA,TSLA game over,Neutral,0.0000


In [6]:
tweet_df.groupby(['Ticker', 'Sentiment']).agg({'Tweet':'count','Score':'mean'})

Tweet     Score
Ticker Sentiment                 
AAPL   Negative       3 -0.455833
       Neutral       14  0.000000
       Positive      13  0.432877
AMZN   Negative       9 -0.444733
       Neutral       12  0.000000
       Positive       9  0.498089
FB     Negative       5 -0.438740
       Neutral       11  0.000000
       Positive      14  0.541629
GOOG   Negative       5 -0.277960
       Neutral       15  0.000000
       Positive      10  0.483480
MSFT   Negative       5 -0.302880
       Neutral       18  0.000000
       Positive       7  0.623514
TSLA   Negative       7 -0.471586
       Neutral       13  0.000000
       Positive      10  0.486170